In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from gensim.models import Doc2Vec
from tensorflow.keras import models
from trec.trectopics import TrecTopics
from arcs.data_generator import DataGenerator
from tensorflow.keras.backend import l2_normalize
from utils import docs_filter

from tensorflow import keras
keras.backend.set_learning_phase(0)

print(keras.__version__)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
2.4.0


Load all necessary models

In [2]:
d2v_model_file = "f:/Models/Lemma_DBOW_2_doc2vec_trec_d1000_n5_w8_mc20_t12_e10_dbow.model"

# test cosine similarity
retrieval_sbm_model = "f:/Models/NN/sbm_predict_cos"
topics_path = "f:/Runs/topics"

In [3]:
d2v = Doc2Vec.load(d2v_model_file)
sbm = models.load_model(retrieval_sbm_model)
topics = TrecTopics(topics_path=topics_path)

# vectorize TREC queries
topics.vectorize(vocab_dict=d2v.wv.vocab,
                 include_title=True, include_desc=False, include_narr=False)

Create data generator for retrieval SBM model prediction

In [4]:
topic_no = 51  # <===== target query number

valid = ["AP88", "AP89", "DOE", "FR88", "FR89", "WSJ87", "WSJ88", "WSJ89", "WSJ90", "WSJ91", "WSJ92", "ZF1", "ZF2"]
valid_docs = docs_filter(d2v.docvecs.index2entity, valid)

docvecs = [d2v.docvecs[doc] for doc in valid_docs]

# docvecs = d2v.docvecs.vectors_docs
targer_vecs = topics.get_topic_vector(topic_no)

data_feeder = DataGenerator(docvecs, targer_vecs, shuffle=False, batch_size=128, sec_input_const=True)

Compute cosine similarity of all document with particular topic query

In [5]:
# import tensorflow as tf

# ttt = data_feeder.__getitem__(1)
# with tf.device("cpu:0"):
cossim = sbm.predict(x=data_feeder, verbose=1)
# cossim = sbm(inputs=data_feeder, training=False, verbose=1)

5764/5764 [==============================] - 4506s 782ms/step

Sort document by descending order of cosine similarity

In [6]:
# docno_list = d2v.docvecs.index2entity
result = dict(zip(valid_docs, cossim))

sorted_result = sorted(result.items(), key=lambda x: x[1], reverse=True)

Create run file

In [7]:
# valid = ["AP88", "AP89", "DOE", "FR88", "FR89", "WSJ87", "WSJ88", "WSJ89", "WSJ90", "WSJ91", "WSJ92", "ZF1", "ZF2"]
#
# def create_trec_runs(runs_file, rank_dict, topic_no, top=-1):
#     count = 0
#     sep = " "
#     with open(runs_file, 'w') as fp:
#         for key, value in rank_dict:
#             if top is not -1 and count >= top:
#                 return
#             for prefix in valid:
#                 if key.startswith(prefix):
#                     fp.write(sep.join([str(topic_no), "Q0", key, str(count), str(value[0]), "test", "\n"]))
#                     count += 1
#                     break
            # count += 1
from utils import create_trec_runs

runs_file = r"f:\Runs\51_cos.txt"
create_trec_runs(runs_file, sorted_result, topic_no, top=-1)


In [28]:
doc = d2v.docvecs["ZF109-514-515"]

cos = sbm.predict(x=[doc.reshape((1,-1)), targer_vecs.reshape(1,-1)])

print(cos)

[[0.0177303]]


In [8]:
import pickle

result_file = r"f:/Runs/results_cos.data"

with open(result_file, 'wb') as fp:
    pickle.dump(sorted_result, fp)

In [ ]:
keyword_model = r"f:/Models/NN/sbm_keyword"

keyword = models.load_model(keyword_model)

doc = d2v.docvecs["WSJ920227-0147"]

vec = keyword.predict(x=doc)